# CNN-LSTM Hybrid

Note: All folders have been saved in runtime or Google Drive

In [1]:
# Importing Drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install noisereduce
!pip install pydub
!pip install torchmetrics

import torchvision.models as models
import os
from PIL import Image
import numpy as np
import collections
from datetime import datetime
import time
import librosa
import sys
import matplotlib.pyplot as plt
import pandas as pd
import io
from tqdm import tqdm
import librosa
import librosa.display
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.signal import butter, lfilter
import json
import librosa.feature
from contextlib import nullcontext
import torch
import torch.nn as nn
from torch.utils.data import random_split, DataLoader, TensorDataset, WeightedRandomSampler
from torchvision import transforms
import noisereduce as nr
from pydub import AudioSegment, silence
from pydub.silence import detect_nonsilent
import torchmetrics
import torch.optim
import torch.optim as opt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
!unzip '/content/spectrogram_dataset.zip'

Streaming output truncated to the last 5000 lines.
  inflating: spectrogram_dataset/negative/a5fa6e70-f8bb-4b84-93fa-c153b03514f5.png  
  inflating: __MACOSX/spectrogram_dataset/negative/._a5fa6e70-f8bb-4b84-93fa-c153b03514f5.png  
  inflating: spectrogram_dataset/negative/e39a3d8b-8143-40e3-a991-d7c7a8aef8b8.png  
  inflating: __MACOSX/spectrogram_dataset/negative/._e39a3d8b-8143-40e3-a991-d7c7a8aef8b8.png  
  inflating: spectrogram_dataset/negative/c3542347-5c21-4892-819c-c30853e877a1.png  
  inflating: __MACOSX/spectrogram_dataset/negative/._c3542347-5c21-4892-819c-c30853e877a1.png  
  inflating: spectrogram_dataset/negative/Ahv0311uRogDaFFI4X22nzsNVvK2_1.png  
  inflating: __MACOSX/spectrogram_dataset/negative/._Ahv0311uRogDaFFI4X22nzsNVvK2_1.png  
  inflating: spectrogram_dataset/negative/c7d13aae-05a9-415b-b0e2-045667d0a5af.png  
  inflating: __MACOSX/spectrogram_dataset/negative/._c7d13aae-05a9-415b-b0e2-045667d0a5af.png  
  inflating: spectrogram_dataset/negative/e7006acb-7872-

# Model

In [147]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNLSTM(nn.Module):
    def __init__(self):
        super(CNNLSTM, self).__init__()

        # CNN for feature extraction
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16, momentum=0.01),  # Lower momentum for stable running stats
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32, momentum=0.01),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, momentum=0.01),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, momentum=0.01),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Calculate CNN output size (assuming 224x224 input)
        self.cnn_output_size = 256 * 7 * 7  # 12544

        # LSTM parameters
        self.lstm_hidden_size = 128
        self.lstm_num_layers = 2

        # Fully connected layer to transform CNN features to LSTM input
        self.fc_before_lstm = nn.Linear(self.cnn_output_size, self.lstm_hidden_size)

        # LSTM
        self.lstm = nn.LSTM(
            input_size=self.lstm_hidden_size,
            hidden_size=self.lstm_hidden_size,
            num_layers=self.lstm_num_layers,
            batch_first=True
        )

        # Final classifier
        self.fc = nn.Sequential(
            nn.Linear(self.lstm_hidden_size, 100),
            nn.ReLU(),
            nn.Linear(100, 1)
        )

    def forward(self, x):
        batch_size = x.size(0)

        # CNN feature extraction
        cnn_features = self.cnn(x)  # (batch_size, 256, 7, 7)
        cnn_features = cnn_features.view(batch_size, -1)  # (batch_size, 12544)

        # Prepare sequence for LSTM
        transformed_features = self.fc_before_lstm(cnn_features)  # (batch_size, lstm_hidden_size)
        lstm_input = transformed_features.unsqueeze(1)  # (batch_size, 1, lstm_hidden_size)

        # Initialize hidden state and cell state
        h0 = torch.zeros(self.lstm_num_layers, batch_size, self.lstm_hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm_num_layers, batch_size, self.lstm_hidden_size).to(x.device)

        # LSTM
        lstm_output, _ = self.lstm(lstm_input, (h0, c0))  # lstm_output: (batch_size, 1, lstm_hidden_size)

        # Use the last output for classification
        last_output = lstm_output[:, -1, :]  # Take the last timestep output

        # Classification
        out = self.fc(last_output)  # (batch_size, 1)

        return out


# DataPipeline

In [144]:
class DataPipeline:
    """Processes datasets, including loading, splitting, and preparing for inference.

    This class provides methods for loading datasets, processing them for training,
    and preparing single instances for inference.

    Attributes:
        test_size (float): Proportion of the dataset to include in the test split.
        val_size (float): Proportion of the dataset to include for validation.
    """

    def __init__(self, test_size: float, val_size: float):
        """Initializes the DatasetProcessor.

        Args:
            data_path (str): Path to the dataset file.
            test_size (float): Proportion of the dataset to include in the test split.
        """
        self.test_size = test_size
        self.val_size = val_size

    def load_dataset(self) -> TensorDataset:
        """Loads the dataset from the specified file path into a DataFrame."""
        tensors = []
        labels = []

        for label_folder, label_value in zip(["positive", "negative", "augmented_positive"], [1, 0, 1]):
            spectrogram_folder = '/content/spectrogram_dataset'
            output_dir = os.path.join(spectrogram_folder, label_folder)

            for image_name in tqdm(os.listdir(output_dir)):
                image_path = os.path.join(output_dir, image_name)
                if(image_path[-4:] == '.png'):
                  image_tensor = self.image_to_tensor(image_path)

                  tensors.append(image_tensor)
                  labels.append(label_value)

        # Tensor of all features (N x D) - N is number of samples (377), D is feature dimension (3,224,224)
        X = torch.stack(tensors)
        # Tensor of all labels (N x 1) - 377x1
        y = torch.tensor(labels, dtype=torch.long)

        return TensorDataset(X, y)


    def image_to_tensor(self, image_path: str) -> torch.Tensor:
        """Converts a spectrogram image to a PyTorch tensor.

        Args:
            image_path (str): Path to the spectrogram image file.

        Returns:
            torch.Tensor: The PyTorch tensor representation of the image.
        """
        transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize to ResNet18 input size
            transforms.ToTensor(),  # Convert image to tensor
        ])

        image = Image.open(image_path).convert("RGB") # Convert from RGBA to RGB
        tensor_image = transform(image)

        return tensor_image  # shape will be 3, 224, 224

    def create_dataloaders(self, batch_size, dataset_path = None, upsample = False) -> tuple[DataLoader, DataLoader, DataLoader]:
        """Splits the dataset into training and test sets.

        Args:
            batch_size (int): The batch size for the DataLoader.
            dataset_path (str | None): Path to the TensorDataset file.

        Returns:
            tuple: (train_df, test_df) - The training and testing DataFrames.
        """
        if dataset_path:
            print(f"Loading dataset from {dataset_path}")
            dataset = torch.load(dataset_path, weights_only=False)
        else:
            print("Processing and loading dataset")
            dataset = self.load_dataset()

        # Calculate sizes
        test_size = round(self.test_size * len(dataset))
        val_size = round(self.val_size * len(dataset))
        train_size = round(len(dataset) - test_size - val_size)  # Remaining for training

        # Perform split
        train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

        # Upsample positive class
        if upsample:
            print("Upsampling data")
            labels = [label.item() for _, label in train_dataset]
            train_counts = {}
            for label in labels:
                train_counts[label] = train_counts.get(label, 0) + 1
            # print(train_counts)

            weights = torch.where(torch.tensor(labels) == 0, 1 / train_counts[0], 1 / train_counts[1])
            # print(labels[:5], weights[:5])

            wr_sampler = WeightedRandomSampler(weights, int(len(train_dataset) * 1.5))

            train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=wr_sampler)

        else:
            print("No upsampling")
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Create DataLoaders
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Count labels in train_loader
        train_counts = {}
        for _, labels in train_loader:
            for label in labels:
                train_counts[label.item()] = train_counts.get(label.item(), 0) + 1

        print(train_counts)

        # Reduce memory footprint
        dataset, train_dataset, val_dataset, test_dataset = None, None, None, None

        return train_loader, val_loader, test_loader

# Model Handler

In [145]:
class ModelHandler:
    """Handles the model training, evaluation, and inference pipeline.

    Attributes:
        device (torch.device): The device on which the model is executed (e.g., 'cpu' or 'cuda').
        model_path: Path to where .pth models should be saved.
    """

    def __init__(
        self,
        model: nn.Module,
        model_path: str,
        optimizer: opt.Optimizer,
        loss_function: nn.Module,
        lr_scheduler: opt.lr_scheduler.LRScheduler,
    ):
        """Initializes the ModelHandler.

        Args:
            model (nn.Module): The machine learning model to be trained/evaluated.
            model_path (str | None): Path to the pre-trained model file (if available).
            optimizer (torch.optim.Optimizer): The optimizer used for training the model.
            loss_function (nn.Module): The loss function used for training the model.
            lr_scheduler (torch.optim.lr_scheduler.LRScheduler): The learning rate scheduler.

        Example Usage:
            model = CNNModel()
            optimizer = opt.Adam(model.parameters(), lr=0.001)
            loss_function = nn.BCEWithLogitsLoss()
            lr_scheduler = opt.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
            model_handler = ModelHandler(model, model_path, optimizer, loss_function, lr_scheduler)
        """
        self.model = model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model_path = model_path
        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler
        self.loss_function = loss_function

    def train_step(self, dataloader) -> dict:
        """Used by self.train(). Trains the model for a single epoch.

        Args:
            dataloader (torch.utils.data.DataLoader): DataLoader for the training dataset.

        Returns:
            Dictionary of training information.
                "avg_loss_per_batch": Average loss per batch.
                "avg_acc_per_batch": Average accuracy per batch.
        """
        self.model.train()  # Set model to training mode
        avg_loss, acc = (
            0,
            0,
        )  # We will calculate the average loss and accuracy per batch
        for in_tensor, labels in dataloader:
            in_tensor, labels = in_tensor.to(self.device), labels.to(self.device)
            labels = labels.float().unsqueeze(1)  # Ensure correct shape for BCE loss

            logits = self.model(in_tensor)  # Feed input into model

            loss = self.loss_function(logits, labels)  # Calculate batch loss
            avg_loss += loss.item()  # Add to cumulative loss

            # Gradient descent
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            # Calculate batch accuracy and add it to cumulative accuracy
            prediction_classes = torch.round(torch.sigmoid(logits))
            batch_acc = torch.mean((prediction_classes == labels).float()).item()
            acc += batch_acc

        avg_loss /= len(dataloader)  # Calculate avg loss for epoch from cumulative loss
        acc /= len(
            dataloader
        )  # Calculate avg accuracy for epoch from cumulative accuracy
        train_results = {"avg_loss_per_batch": avg_loss, "avg_acc_per_batch": acc * 100}
        return train_results

    def val_step(self, dataloader) -> dict:
        """Used by self.train(). Evaluates the model on the validation dataset.

        Args:
            dataloader (torch.utils.data.DataLoader): DataLoader for the validation dataset.

        Returns:
            Dictionary of validation information.
                "avg_loss_per_batch": Average loss per batch.
                "avg_acc_per_batch": Average accuracy per batch.
        """

        self.model.eval()
        with torch.inference_mode():
            avg_loss, acc = 0, 0
            for in_tensor, labels in dataloader:
                in_tensor, labels = in_tensor.to(self.device), labels.to(self.device)
                labels = labels.float().unsqueeze(
                    1
                )  # Ensure correct shape for BCE loss

                logits = self.model(in_tensor)  # Feed input into model

                loss = self.loss_function(logits, labels)  # Calculate batch loss
                avg_loss += loss.item()  # Add to cumulative loss

                # Calculate batch accuracy and add it to cumulative accuracy
                prediction_classes = torch.round(torch.sigmoid(logits))
                batch_acc = torch.mean((prediction_classes == labels).float()).item()
                acc += batch_acc

            avg_loss /= len(
                dataloader
            )  # Calculate avg loss for each epoch from cumulative loss
            acc /= len(
                dataloader
            )  # Calculate avg accuracy for each epoch from cumulative accuracy
            valid_results = {
                "avg_loss_per_batch": avg_loss,
                "avg_acc_per_batch": acc * 100,
            }
            return valid_results

    def train(
        self, train_loader, val_loader, epochs: int, model_name: str
    ) -> tuple[dict, dict]:
        """Trains the model.

        Args:
            train_loader: DataLoader for the training datasets
            epochs (int): Number of training epochs.
            model_name (str): Name to save the trained model.

        Returns:
            Two dictionaries containing the following training and validation information:
                "epoch": List of epoch numbers.
                "loss": List of average loss per batch.
                "accuracy": List of average accuracy per
        """
        self.model.to(self.device)
        training_results = {"epoch": [], "loss": [], "accuracy": []}
        validation_results = {"epoch": [], "loss": [], "accuracy": []}

        for epoch in range(epochs):

            # Train the model
            training_data = self.train_step(train_loader)
            training_results["epoch"].append(epoch)
            training_results["loss"].append(training_data["avg_loss_per_batch"])
            training_results["accuracy"].append(training_data["avg_acc_per_batch"])

            # Check the validation loss after training
            validation_data = self.val_step(val_loader)
            validation_results["epoch"].append(epoch)
            validation_results["loss"].append(validation_data["avg_loss_per_batch"])
            validation_results["accuracy"].append(validation_data["avg_acc_per_batch"])

            # Adjust learning rate if necessary
            if self.lr_scheduler:
                # Some LR schedulers take validation loss as input, others will ignore it (I think)
                self.lr_scheduler.step(validation_data["avg_loss_per_batch"])

            print(f"{epoch}:")
            print(f"LR: {self.optimizer.param_groups[0]['lr']}")
            print(
                f"Loss - {training_data['avg_loss_per_batch']:.5f} | Accuracy - {training_data['avg_acc_per_batch']:.2f}%"
            )
            print(
                f"VLoss - {validation_data['avg_loss_per_batch']:.5f} | VAccuracy - {validation_data['avg_acc_per_batch']:.2f}%\n"
            )
        self.save_model(model_state_dict=self.model.state_dict(), model_name=model_name)
        return training_results, validation_results

    def validate(
        self, val_loader, hyperparams: dict, save_best: bool = True
    ) -> tuple[float, float]:
        """Validates the model on the validation dataset.

        Args:
            val_loader: DataLoader for the validation dataset.

        Returns:
            tuple: (validation accuracy, validation loss)
        """

        self.model.to(self.device)
        self.model.eval()

        val_losses_epoch, batch_sizes, accs = [], [], []
        best_acc = -1
        best_model_state = None  # Track the best model weights

        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val = X_val.to(self.device)
                y_val = y_val.to(self.device).float().unsqueeze(1)

                y_prediction_val = self.model(X_val)  # forward pass
                loss = self.loss_function(y_prediction_val, y_val)
                val_losses_epoch.append(loss.item())

                # Compute accuracy
                y_prediction_val = torch.sigmoid(
                    y_prediction_val
                )  # Convert logits to probabilities
                prediction_classes = (
                    y_prediction_val > 0.5
                ).float()  # Convert to binary 0/1

                acc = torch.mean((prediction_classes == y_val).float()).item()
                accs.append(acc)
                batch_sizes.append(X_val.shape[0])

        # Compute final validation loss and accuracy
        val_loss = np.mean(val_losses_epoch)
        val_acc = np.average(accs, weights=batch_sizes)  # Weighted average accuracy

        print(
            f"Validation accuracy: {val_acc*100:.2f}% | Validation loss: {val_loss:.4f}"
        )

        if save_best and val_acc > best_acc:
            best_acc = val_acc
            best_model_state = self.model.state_dict()

            # Create model filename using hyperparameters
            hyperparam_str = "_".join(
                f"{key}:{value}" for key, value in hyperparams.items()
            )
            model_filename = f"model_{hyperparam_str}_{time.time()}.pth"

            # Save the best model
            save_path = os.path.join(self.model_path, model_filename)
            torch.save(best_model_state, save_path)
            print(f"Best model saved at: {save_path}")
        return val_acc, val_loss

    def evaluate(self, test_loader) -> float:
        """Evaluates the model on the test dataset.

        Args:
            test_loader: DataLoader for the test dataset.
        """
        self.model.to(self.device)
        self.model.eval()
        batch_sizes, accs = [], []
        with torch.no_grad():
          for (
              X_test,
              y_test,
          ) in test_loader:
              X_test = X_test.to(self.device)
              y_test = y_test.to(self.device).float().unsqueeze(1)

              prediction = self.model(X_test)
              batch_sizes.append(X_test.shape[0])

              prediction = torch.round(torch.sigmoid(prediction))
              # prediction_classes = (
              #     prediction > 0.5
              # ).float()  # This converts to binary classes 0 and 1

              acc = torch.mean((prediction == y_test).float()).item()
              accs.append(acc)

        # Return average accuracy
        print(accs)
        return np.average(accs, weights=batch_sizes)

    def predict(self, spectrogram: torch.Tensor) -> int:
        """Performs inference on a single spectrogram.

        Args:
            spectrogram (torch.Tensor): Input spectrogram for inference.

        Returns:
            torch.Tensor: The predicted output from the model.
        """

        self.model.load_state_dict(
            torch.load(self.model_path, map_location=torch.device("cpu"))
        )
        self.model.to(self.device)
        self.model.eval()

        spectrogram = spectrogram.unsqueeze(0).to(self.device)

        with torch.no_grad():
            logits = self.model(spectrogram)

            probability = torch.sigmoid(logits)
            print(probability)

            prediction = (
                probability > 0.5
            ).float()  # Turn probability into binary classificaiton
        print("Performed prediction on image.")
        return prediction.item()

    def save_model(
        self, model_state_dict: collections.OrderedDict, model_name: str | None
    ) -> None:
        """Saves the model to the specified file path.

        Args:
            path (str): Path to save the model file.
        """
        path = self.model_path + "/" + model_name
        torch.save(model_state_dict, path)

    def load_model(self, path: str) -> None:
        """Loads a model from the specified file path.

        Args:
            path (str): Path to the model file.
        """
        self.model.load_state_dict(torch.load(path))
        self.model.to(self.device)
        self.model.eval()

# Dataset

In [8]:
datapipeline = DataPipeline(test_size=0.15, val_size=0.15)
train_loader, val_loader, test_loader = datapipeline.create_dataloaders(batch_size=8)

Processing and loading dataset


100%|██████████| 1164/1164 [00:12<00:00, 90.31it/s] 


No upsampling
{0: 2174, 1: 2178}


# Training

In [148]:
# Instantiate the model
model = CNNLSTM()

# Define optimizer, loss function, and learning rate scheduler
optimizer = opt.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)  # SGD optimizer
# optimizer = opt.Adam(model.parameters(), lr=0.001)
loss_function = nn.BCEWithLogitsLoss()
lr_scheduler = opt.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Define model save path
model_path = "/content/Models"

# Instantiate the model handler
model_handler = ModelHandler(
    model=model,
    model_path=model_path,
    optimizer=optimizer,
    loss_function=loss_function,
    lr_scheduler=lr_scheduler,
)

# Set training parameters
epochs = 15
model_name = "krish_cnn_lstm_hybrid.pth"

# Train the model
training_results, validation_results = model_handler.train(train_loader, val_loader, epochs, model_name)

# Evaluate on test set
test_accuracy = model_handler.evaluate(test_loader)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

0:
LR: 0.001
Loss - 0.69301 | Accuracy - 51.47%
VLoss - 0.69265 | VAccuracy - 56.94%

1:
LR: 0.001
Loss - 0.69214 | Accuracy - 54.55%
VLoss - 0.69138 | VAccuracy - 58.01%

2:
LR: 0.001
Loss - 0.69009 | Accuracy - 56.59%
VLoss - 0.68750 | VAccuracy - 62.07%

3:
LR: 0.001
Loss - 0.68328 | Accuracy - 62.13%
VLoss - 0.67453 | VAccuracy - 66.45%

4:
LR: 0.001
Loss - 0.65839 | Accuracy - 66.25%
VLoss - 0.63050 | VAccuracy - 69.55%

5:
LR: 0.001
Loss - 0.60722 | Accuracy - 69.19%
VLoss - 0.67141 | VAccuracy - 61.75%

6:
LR: 0.001
Loss - 0.53631 | Accuracy - 74.29%
VLoss - 0.50451 | VAccuracy - 78.21%

7:
LR: 0.001
Loss - 0.49209 | Accuracy - 77.18%
VLoss - 0.50888 | VAccuracy - 75.64%

8:
LR: 0.001
Loss - 0.47999 | Accuracy - 78.12%
VLoss - 0.46150 | VAccuracy - 79.38%

9:
LR: 0.001
Loss - 0.46569 | Accuracy - 78.19%
VLoss - 0.46188 | VAccuracy - 79.70%

10:
LR: 0.001
Loss - 0.44569 | Accuracy - 79.50%
VLoss - 0.47008 | VAccuracy - 78.21%

11:
LR: 0.001
Loss - 0.43656 | Accuracy - 80.33%
VLos